In [45]:
import json
import pandas as pd
from os.path import join
from scipy.io import savemat
import numpy as np
import itertools
from surgeon_recording.reader import Reader
from scipy.io import savemat
from sklearn.preprocessing import MinMaxScaler

In [12]:
reader = Reader()

In [15]:
#datafolder = join('..', 'data', 'cuts')
datafolder = join('F:\PDM\data','cuts') #data sur disque dur

In [37]:
nb_files = 20
exp_folders = [join(datafolder, 'cut' + str(i)) for i in (np.arange(nb_files) + 2)]   #prend les vaeurs datafolder/cuti avec i de 2 à 21

In [38]:
exp_folders

['F:\\PDM\\data\\cuts\\cut2',
 'F:\\PDM\\data\\cuts\\cut3',
 'F:\\PDM\\data\\cuts\\cut4',
 'F:\\PDM\\data\\cuts\\cut5',
 'F:\\PDM\\data\\cuts\\cut6',
 'F:\\PDM\\data\\cuts\\cut7',
 'F:\\PDM\\data\\cuts\\cut8',
 'F:\\PDM\\data\\cuts\\cut9',
 'F:\\PDM\\data\\cuts\\cut10',
 'F:\\PDM\\data\\cuts\\cut11',
 'F:\\PDM\\data\\cuts\\cut12',
 'F:\\PDM\\data\\cuts\\cut13',
 'F:\\PDM\\data\\cuts\\cut14',
 'F:\\PDM\\data\\cuts\\cut15',
 'F:\\PDM\\data\\cuts\\cut16',
 'F:\\PDM\\data\\cuts\\cut17',
 'F:\\PDM\\data\\cuts\\cut18',
 'F:\\PDM\\data\\cuts\\cut19',
 'F:\\PDM\\data\\cuts\\cut20',
 'F:\\PDM\\data\\cuts\\cut21']

In [18]:
def insert_row(data, row, labels=None):
    return data.append(pd.Series(row, labels), ignore_index=True)          #ajoute a data le vecteur avec ses axis labels    

#ignore_index= true: the resulting axis are labeled 0,1,...,n-1

In [25]:
def preprocess_motion(data, frames_list, scaler):                          # DATA STANDARDIZATION, scaler peut être plusieurs choses différentes (soit faire en sorte que les data soient entre [0,1] ou [-1,1]....
    column_list = list(itertools.chain.from_iterable((f + '_x', f + '_y', f + '_z') for f in frames_list))  
    data[column_list] = 2 * scaler.fit_transform(data[column_list]) - 1    #fit transform with the chosen scaler, et *2 -1
    
    #f prend les valeurs de frame_list données dans la fonction
    #on obtient f_x, f_y, f_z
    #list: donne la liste de tous les noms (sépare pas quand il y a l'opérateur +) donc: ['f1_x','f1_y','f1_z','f2_x','f2_y',...] 
    #permet d'accéder aux data qui ont les labels

In [26]:
#frames_list=['1','2','3','4','5']
#from_iterable = itertools.chain.from_iterable((f + '_x', f + '_y', f + '_z') for f in frames_list)
#print(list(from_iterable))

In [27]:
def downsample(data, time_vector):
    current_time_index = 0
    downsampled_data = pd.DataFrame(columns=data.columns)
    
    for i in range(len(time_vector)):
        t = time_vector.iloc[i]             #t prend la ieme valeur du vecteur de temps
        start_time = current_time_index     #part de 0, puis part du dernier stop time quand boucle
        
        while current_time_index < data.shape[0] and data['relative_time'].iloc[current_time_index] < t:   #pour trouver stop time: tant que la valeur a la position de current_index_time dans la colonne relative time des data est plus petite que la ieme valeur du vecteur de temps ET qu'on n'est pas arrivé au bout de la dimension de data
            current_time_index = current_time_index + 1
        stop_time = current_time_index

        average_data = data.iloc[start_time:stop_time, :].mean()         #calcule la moyenne des data  de start time at stop time
        downsampled_data = insert_row(downsampled_data, average_data)    #ajoute les lignes  à la suite avec la fonction spécifique 
    return downsampled_data

#PERMET DE CALCULER LES VALEURS POUR N?IMPORTE QUEL VECTEUR DE TEMPS AVEC LES VALEURS MOYENNES CALCULEES SI ON A PLUSIEURS POINTS DE MESURES COMPRIS ENTRE 2 VALEURS DU VECTEUR DE TEMPS

In [29]:
def preprocess_emg(data, emg_list, scaler):
    # first take the absolute value of the data
    data[emg_list] = data[emg_list].abs()
    # then apply a minmax scaler
    data[emg_list] = 2 * scaler.fit_transform(data[emg_list]) - 1
    
    #DATA STANDARDIZATION,

In [31]:
frames_list = ['blade', 'wrist']
optitrack_list = list(itertools.chain.from_iterable((f + '_x', f + '_y', f + '_z',
                                                     f + '_qx', f + '_qy', f + '_qz', f + '_qw')
                                                    for f in frames_list))                         #donne ['blade_x','blade_y','blade_z','blade_qx...]
emg_list = ['emg' + str(i) for i in range(8)]                                                      #emg0, emg1...emg7

In [36]:
timeseries = []

scalers = {}
scalers['optitrack'] = MinMaxScaler()           #définit les scaler utilisés pour standardiser les mesures
scalers['emg'] = MinMaxScaler()                 #MinMaxScaler: X_std = (X - X.min(axis=0)) / (X.max(axis=0) - X.min(axis=0)), X_scaled = X_std * (max - min) + min
                                                #Transform features by scaling each feature to a given range. This estimator scales and translates each feature individually such that it is in the given range on the training set, e.g. between zero and one.

for folder in exp_folders:
    reader.play(folder)
    # extract optitrack data and preprocess them
    opt_data = reader.data['optitrack'].reset_index(drop=True)               #enlève les index numériques des data
    preprocess_motion(opt_data, frames_list, scalers['optitrack'])           #foction preprocess pour standardizer
    # downsample emg and preprocess it
    emg_data = downsample(reader.data['emg'], opt_data['relative_time'])     #downsample les emg data pouravoir meme sampling time et time vector que les optitrack data !!!!!!!!!
    preprocess_emg(emg_data, emg_list, scalers['emg'])                       #standardisation emg
    # merge the data
    merge_data = pd.concat([opt_data, emg_data.iloc[:, 3:]], axis=1)         #axis=1: permet de garder sur la meme ligne = on ajoute les valeurs en dessous
    # store in the list
    timeseries.append(merge_data)

In [44]:
#vector=np.array([1,2,3,4,5,6,7,8,8,9])
#vector.iloc[:, 3:]


# Save as mat file for Nadia's segmentation

In [ ]:
merged_column_list = optitrack_list

# drop the data in matlab
matdata = {}
matdata['timeseries'] = []
for t in timeseries:
    # drop the index and times columns and convert to numpy array
    matdata['timeseries'].append(t[merged_column_list].to_numpy())

In [ ]:
savemat("segmentation_data.mat", matdata)

# Apply 0 velocity segmentation

## Compute velocities

In [ ]:
for i, data in enumerate(timeseries):
    frame_list = ['blade', 'wrist']

    header = list(itertools.chain.from_iterable((f + '_dx', f + '_dy', f + '_dz', f + '_vel_mag',
                                                 f + '_ddx', f + '_ddy', f + '_ddz', f + '_acc_mag')
                                                for f in frame_list))

    derived_data = pd.DataFrame(columns=header)

    for index, row in data.iterrows():
        row_data = []
        for f in frame_list:
            # compute velocity
            if index == 0:
                vel = np.zeros(3)
            else:
                x0 = data.loc[index - 1, [f + '_x', f + '_y', f + '_z']].to_numpy()
                x1 = row.loc[[f + '_x', f + '_y', f + '_z']].to_numpy()
                dt = row['relative_time'] - data.loc[index - 1, 'relative_time']
                vel = (x1 - x0) / dt
            vel_mag = np.linalg.norm(vel)
            # compute accleration
            if index < 2:
                acc = np.zeros(3)
            else:
                x0 = data.loc[index - 2, [f + '_x', f + '_y', f + '_z']].to_numpy()
                x1 = data.loc[index - 1, [f + '_x', f + '_y', f + '_z']].to_numpy()
                x2 = row.loc[[f + '_x', f + '_y', f + '_z']].to_numpy()
                dt = (row['relative_time'] - data.loc[index - 2, 'relative_time']) / 2.
                acc = (x2 - 2*x1 + x0) / (dt *  dt)
            acc_mag = np.linalg.norm(acc)
            frame_vector = np.hstack((vel, vel_mag, acc, acc_mag))
            row_data = np.hstack((row_data, frame_vector))
        derived_data = insert_row(derived_data, row_data, header)
        timeseries[i] = pd.concat([data, derived_data], axis=1)
        
        
        #to segment based on 0 velocity data = cut each time you have 0 velocity

## compute normed and rolling average

In [ ]:
vel_frames = list(itertools.chain.from_iterable((f + '_vel_mag', f + '_acc_mag') for f in frame_list))

In [ ]:
window_size = 10

In [ ]:
normed_velocities = []
rolling_average_velocities = []

for data in timeseries:
    vel_acc_data = data.loc[:, vel_frames]
    normed_velocities.append((vel_acc_data - vel_acc_data.min()) / (vel_acc_data.max() - vel_acc_data.min()))
    rolling_average_velocities.append(normed_velocities[-1].rolling(window=window_size).mean().iloc[window_size-1:])

In [ ]:
idx = 1
normed_velocities[idx].plot.line()
rolling_average_velocities[idx].plot.line()